In [16]:
import pandas as pd


clients = pd.read_csv("datos/caso_1/clients.csv")
depots = pd.read_csv("datos/caso_1/depots.csv")
depots = depots.drop(columns=["DepotID"])
vehicles = pd.read_csv("datos/caso_1/vehicles.csv")


print(clients.head())
print(depots.head())
print(vehicles.head())


   ClientID  LocationID  Demand  Longitude  Latitude
0         1           2      13 -74.098938  4.597954
1         2           3      15 -74.075571  4.687821
2         3           4      12 -74.107085  4.709494
3         4           5      15 -74.097280  4.605029
4         5           6      20 -74.164641  4.648464
   LocationID  Longitude  Latitude
0           0 -74.153536  4.743359
   VehicleID  Capacity  Range
0          1       130    170
1          2       140    200
2          3       120    180
3          4       100     90
4          5        70    100


In [32]:

depots = depots.rename(columns={"LocationID": "id"})
clients = clients.rename(columns={"LocationID": "id"})
depots["id"] = 0 
depots["Demand"] = 0

clients = clients.reset_index(drop=True)
clients["id"] = clients.index + 1  

depots = depots[["id", "Longitude", "Latitude", "Demand"]]
clients = clients[["id", "Longitude", "Latitude", "Demand"]]

nodes = pd.concat([depots, clients], ignore_index=True)


N = nodes["id"].tolist()

C = clients["id"].tolist()

De = nodes.set_index("id")["Demand"].to_dict()

depot = 0

V = vehicles["VehicleID"].tolist()
P = vehicles.set_index("VehicleID")["Capacity"].to_dict()
M = vehicles.set_index("VehicleID")["Range"].to_dict()

print("Nodos:", N)
print("Clientes:", C)
print("Vehículos:", V)
print("Capacidades:", P)
print("Alcances:", M)
print("Demanda:", De)



Nodos: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Clientes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Vehículos: [1, 2, 3, 4, 5, 6, 7, 8]
Capacidades: {1: 130, 2: 140, 3: 120, 4: 100, 5: 70, 6: 55, 7: 110, 8: 114}
Alcances: {1: 170, 2: 200, 3: 180, 4: 90, 5: 100, 6: 170, 7: 150, 8: 140}
Demanda: {0: 0, 1: 13, 2: 15, 3: 12, 4: 15, 5: 20, 6: 17, 7: 17, 8: 20, 9: 20, 10: 15, 11: 17, 12: 12, 13: 21, 14: 15, 15: 17, 16: 10, 17: 25, 18: 12, 19: 11, 20: 15, 21: 14, 22: 18, 23: 15, 24: 11}


In [ ]:
import openrouteservice
from openrouteservice import convert
import time  

def get_distancia_y_tiempo(api_key, start_coords, end_coords):
    client = openrouteservice.Client(key=api_key)

    try:
        route = client.directions(
            coordinates=[start_coords, end_coords],
            profile='driving-car',
            format='geojson'
        )

        distance_m = route['features'][0]['properties']['segments'][0]['distance']
        duration_s = route['features'][0]['properties']['segments'][0]['duration']

        distance_km = distance_m / 1000
        duration_min = duration_s / 60

        return distance_km, duration_min
    except Exception as e:
        print(f"Error con ruta {start_coords} -> {end_coords}: {e}")
        return float('inf'), float('inf')  

# Clave API
api_key = '5b3ce3597851110001cf62487e04df114afa4c3ab39b2f9422fc450a'

In [ ]:
distances = {}


coords = nodes.set_index("id")[["Longitude", "Latitude"]].to_dict(orient="index")


for i in coords:
    for j in coords:
        if i != j:
            start = (coords[i]["Longitude"], coords[i]["Latitude"])
            end = (coords[j]["Longitude"], coords[j]["Latitude"])
            dist, dur = get_distancia_y_tiempo(api_key, start, end)

            distances[(i, j)] = dist

            time.sleep(1.2)  
        else:
            distances[(i, j)] = 0.0  

In [36]:
print(distances)

{(0, 0): 0.0, (0, 1): 27.1938, (0, 2): 16.9897, (0, 3): 12.367, (0, 4): 26.7042, (0, 5): 22.637400000000003, (0, 6): 19.6539, (0, 7): 25.0218, (0, 8): 23.8935, (0, 9): 30.672900000000002, (0, 10): 33.8628, (0, 11): 33.4883, (0, 12): 15.8655, (0, 13): 21.6602, (0, 14): 10.360899999999999, (0, 15): 26.7918, (0, 16): 33.9123, (0, 17): 26.608, (0, 18): 19.8704, (0, 19): 20.932599999999997, (0, 20): 13.1411, (0, 21): 27.835099999999997, (0, 22): 24.4072, (0, 23): 11.3399, (0, 24): 30.865299999999998, (1, 0): 30.9567, (1, 1): 0.0, (1, 2): 14.286299999999999, (1, 3): 19.4419, (1, 4): 1.1334000000000002, (1, 5): 12.4978, (1, 6): 11.316799999999999, (1, 7): 17.807599999999997, (1, 8): 13.7538, (1, 9): 7.5148, (1, 10): 7.3902, (1, 11): 10.330200000000001, (1, 12): 25.8113, (1, 13): 8.2964, (1, 14): 20.3067, (1, 15): 1.2928, (1, 16): 7.4397, (1, 17): 4.4865, (1, 18): 9.8418, (1, 19): 14.974200000000002, (1, 20): 23.0869, (1, 21): 21.675900000000002, (1, 22): 5.3789, (1, 23): 21.2856, (1, 24): 4.8